<a href="https://colab.research.google.com/github/sherry975/CSC-791-Project/blob/master/Writting%20Styles%20Embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

# !ls "/content/drive/My Drive/"

import os
os.chdir("/content/drive/My Drive/CSC791-NLP/791 NLP project/791_project_repo")

import matplotlib.pyplot as plt

import pickle

import sys
import traceback

import csv
import numpy as np
from numpy import dot
from numpy.linalg import norm
import pandas as pd
import re

import nltk


In [0]:
#WordtoVec -- GoogleNews
import gensim.models.keyedvectors as word2vec
model = word2vec.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

In [0]:
def wordList2vec(word_list):
    vecs = [model[word] for word in word_list if word in model.vocab]
    ret = np.mean(vecs, axis=0)
    return ret

In [0]:

def pickle_save(filename, data2pkl):
    global pickle_overwrite
    if pickle_overwrite == True:
        fileObject = open(filename,'wb') 
        pickle.dump(data2pkl,fileObject)
        fileObject.close()

        
def pickle_load(filename):
    fileObject = open(filename,'rb') 
    data_unpkl = pickle.load(fileObject)
    fileObject.close()
    return data_unpkl


In [0]:
def build_dictionary(filename):
  rows = pickle_load(filename)
  tokens = {}
  tokens['marked_token'] = []
  tokens['sentence_id'] = []
  tokens['label'] = []
  tokens['token'] = []
  for row in rows:
    marked_token = row[7]
    originate_token = row[6]
    sentence_id = row[0]    
    label = row[5]
    tokens['sentence_id'].append(sentence_id)
    tokens['marked_token'].append([re.sub('[^0-9a-zA-Z]+', ' ', str(marked_token)).lower()])
    tokens['label'].append(label)
    tokens['token'].append([re.sub('[^0-9a-zA-Z]+', ' ', str(originate_token)).lower()])
   
  return tokens
 

In [0]:
_#build model

def evaluation(X_train, Y_train, X_test, Y_test_true):
  from sklearn.linear_model import LogisticRegression
  Y_test_sys = []
  Y_test_proba = []
  model = LogisticRegression(solver='liblinear').fit(X_train, np.ravel(Y_train))
  for k in X_test:
      Y_test_sys.append(model.predict([k]))
      Y_test_proba.append(model.predict_proba([k]))
  results = zip(Y_test_sys, Y_test_proba)
 
  from sklearn.metrics import f1_score
  from sklearn.metrics import confusion_matrix
  from sklearn.metrics import precision_score
  from sklearn.metrics import classification_report
  from sklearn.metrics import accuracy_score
  #f1 score
  print('testing performance feature set ')
  print('f1 score = ', f1_score(Y_test_true, Y_test_sys, average='weighted'))

  #accuracy score

  score = accuracy_score(Y_test_true, Y_test_sys)
  print("accuracy score = ", score)

  #precision score
  print("precision score = ", precision_score(Y_test_true, Y_test_sys, average='weighted'))

  #confusion matrix
  print("confusion matrix = ")
  print(confusion_matrix(Y_test_true, Y_test_sys))

  print('report : ')
  print(classification_report(Y_test_true, Y_test_sys))

In [0]:
nltk.download('punkt')
nltk.download('wordnet')
training_full_data = {}
training_full_data = build_dictionary("output_data/training_masked_list.pkl")
#print(Y)
testing_full_data = {}
testing_full_data = build_dictionary("output_data/testing_masked_list.pkl")

In [0]:

#marked tokens
vectorized_marked = []

i=0
for row in training_full_data['marked_token']:
  #print(row)
  var = ''.join(row)
  vec1 = wordList2vec(var.split())
  if not np.isnan(vec1).any():
    vectorized_marked.append([training_full_data['label'][i], vec1])
 
  i += 1

#print(vectorized)
X_train_marked = np.array([(data[-1]) for data in vectorized_marked])
Y_train_marked = np.array([(data[0]) for data in vectorized_marked])

i=0
vectorized_marked = []
for row in testing_full_data['marked_token']:
  #print(row)
  var = ''.join(row)
  vec1 = wordList2vec(var.split())
  if not np.isnan(vec1).any():
    vectorized_marked.append([testing_full_data['label'][i], vec1])
  i += 1


X_test_marked = np.array([data[-1] for data in vectorized_marked])
Y_test_marked = np.array([data[0] for data in vectorized_marked])

print(len(Y_test_marked))
print(len(X_test_marked))
print(len(Y_train_marked))
print(len(X_train_marked))

print(Y_train_marked)
evaluation(X_train_marked, Y_train_marked, X_test_marked, Y_test_marked)



In [0]:
#originated tokens
vectorized_originated = []

i=0
for row in training_full_data['token']:
  var = ''.join(row)
  vec1 = wordList2vec(var.split())
  if not np.isnan(vec1).any():
    vectorized_originated.append([training_full_data['label'][i], vec1])
    
  i += 1

#print(vectorized)
X_train_originated = np.array([(data[-1]) for data in vectorized_originated])
Y_train_originated = np.array([(data[0]) for data in vectorized_originated])
#print(Y)

i=0
vectorized_originated = []
for row in testing_full_data['token']:
  #print(row)
  var = ''.join(row)
  vec1 = wordList2vec(var.split())
  if not np.isnan(vec1).any():
    vectorized_originated.append([testing_full_data['label'][i], vec1])
  i += 1


X_test_originated = np.array([data[-1] for data in vectorized_originated])
Y_test_originated = np.array([data[0] for data in vectorized_originated])

evaluation(X_train_originated, Y_train_originated, X_test_originated, Y_test_originated)

In [0]:
#ngram
from nltk import word_tokenize 
from nltk.util import ngrams
from gensim.models import Phrases
from gensim.models.phrases import Phraser

#training data
sentences = []

for row in training_full_data['token']:
  var = ''.join(row)
  sentences.append(var)

sentence_stream = [sentence.split(" ") for sentence in sentences]

bigram = Phrases(sentence_stream, min_count=1, threshold=2, delimiter=b' ')
bigram_phraser = Phraser(bigram)
trigram = Phrases(bigram[sentence_stream], min_count=1, threshold=2, delimiter=b' ')
trigram_phraser = Phraser(trigram)


vectorized = []
vectorized1 = []
i = 0
#model = word2vec.Word2Vec(bigram_phraser[sentence_stream], size=300, workers=3)

for word in sentence_stream:
  tokens_ = bigram_phraser[word]
  vec1 = wordList2vec(tokens_)
  tokens_1 = trigram_phraser[word]
  vec2 = wordList2vec(tokens_1)
  #print(tokens_1)
  if not np.isnan(vec1).any():
    vectorized.append([training_full_data['label'][i], vec1])
  if not np.isnan(vec2).any():
    vectorized1.append([training_full_data['label'][i], vec2])
  i += 1

X_train_marked = np.array([(data[-1]) for data in vectorized])
Y_train_marked = np.array([(data[0]) for data in vectorized])
X_train_marked1 = np.array([(data[-1]) for data in vectorized1])
Y_train_marked1 = np.array([(data[0]) for data in vectorized1])
print(model.most_similar('New_York'))

#testing data
sentences = []

for row in testing_full_data['token']:
  var = ''.join(row)
  sentences.append(var)

sentence_stream = [sentence.split(" ") for sentence in sentences]

bigram = Phrases(sentence_stream, min_count=1, threshold=2, delimiter=b' ')
bigram_phraser = Phraser(bigram)
trigram = Phrases(bigram[sentence_stream], min_count=1, threshold=2, delimiter=b' ')
trigram_phraser = Phraser(trigram)

vectorized = []
vectorized1 = []
i = 0
#model = word2vec.Word2Vec(bigram_phraser[sentence_stream], size=300, workers=3)
for word in sentence_stream:
  tokens_ = bigram_phraser[word]
  vec1 = wordList2vec(tokens_)
  tokens_1 = trigram_phraser[word]
  vec2 = wordList2vec(tokens_1)
  #print(tokens_1)
  if not np.isnan(vec1).any():
    vectorized.append([testing_full_data['label'][i], vec1])
  if not np.isnan(vec2).any():
    vectorized1.append([testing_full_data['label'][i], vec2])
  i += 1

X_test_marked = np.array([(data[-1]) for data in vectorized])
Y_test_marked = np.array([(data[0]) for data in vectorized])
X_test_marked1 = np.array([(data[-1]) for data in vectorized1])
Y_test_marked1 = np.array([(data[0]) for data in vectorized1])
print(len(Y_test_marked))
print(len(X_test_marked))
print(len(Y_train_marked))
print(len(X_train_marked))

print(Y_train_marked)
evaluation(X_train_marked, Y_train_marked, X_test_marked, Y_test_marked)
evaluation(X_train_marked1, Y_train_marked1, X_test_marked1, Y_test_marked1)